In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [3]:
#依赖的包
from lib.dataloader import normal_and_generate_dataset_time, get_mask, get_adjacent, get_grid_node_map_maxtrix
#攻击函数import
from lib.utils_new import mask_loss, compute_loss, predict_and_evaluate, attack, random_attack, fgsm_attack, min_attack
from lib.utils_new import saliency, saliency_loss, attack_js, attack_js,attack_saliency,rand_attack
from model.GSNet import GSNet
from lib.early_stop import EarlyStopping
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import json
import configparser
import pickle as pkl
from time import time
from datetime import datetime
import shutil
import argparse
import random
import math

import sys
import os


/home/wyb/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wyb/miniconda3/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
#完整运行时会生成txt日志记录#
train_log_filename = "test_log.txt"
train_log_filepath = os.path.join("./", train_log_filename)
now = datetime.now()
date_time = now.strftime("|%Y-%m-%d, %H:%M:%S| ")

#以命令方式运行时会自动设置工作空间
#file会报错，应该家’‘
curPath = os.path.abspath(os.path.dirname('__file__'))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)

In [6]:
#配置文件载入，初始参数记录

config_filename = "config/nyc/GSNet_NYC_Config.json"
with open(config_filename, 'r') as f:
    config = json.loads(f.read())
# 写入测试超参数
with open(train_log_filepath, "a") as f:
    f.write('测试文件运行时间：'+date_time)
    f.write('\r\n')
    f.write('超参数设置：')
    f.write('\r\n')
    f.write(json.dumps(config, sort_keys=True, indent=4))
    f.write('\r\n')
f.close
#GPU设置
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

#文件读取
#命令行参数实质上就是赋初始值的作用，可以删除直接赋值就好
north_south_map = config['north_south_map']
west_east_map = config['west_east_map']

all_data_filename = config['all_data_filename']
mask_filename = config['mask_filename']

road_adj_filename = config['road_adj_filename']
risk_adj_filename = config['risk_adj_filename']
poi_adj_filename = config['poi_adj_filename']
grid_node_filename = config['grid_node_filename']
grid_node_map = get_grid_node_map_maxtrix(grid_node_filename)
num_of_vertices = grid_node_map.shape[1]


patience = config['patience']#用处：
delta = config['delta']

if config['seed'] is not None:
    seed = config['seed']
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


train_rate = config['train_rate']
valid_rate = config['valid_rate']

recent_prior = config['recent_prior']
week_prior = config['week_prior']
one_day_period = config['one_day_period']
days_of_week = config['days_of_week']
pre_len = config['pre_len']
seq_len = recent_prior + week_prior

training_epoch = config['training_epoch']


True


# 原始数据处理
all_data.pkl       (8760, 48, 20, 20) 8760=365*24
grid_node_map.pkl  (400, 243)  映射矩阵
输出特征大小
训练集：4584*7*48*20*20
验证集：1080*7*48*20*20
测试集：1080*7*48*20*20
7的含义：t =673时刻是，输入7个时间片，[1, 169, 337, 505, 670, 671, 672]

In [7]:
batch_size = config['batch_size']
learning_rate = config['learning_rate']

#模型结构
num_of_gru_layers = config['num_of_gru_layers']
gru_hidden_size = config['gru_hidden_size']
gcn_num_filter = config['gcn_num_filter']

loaders = []
scaler = ""
train_data_shape = ""
graph_feature_shape = ""
   ################## 获取测试数据#########################
for idx, (x, y, target_times, high_x, high_y, high_target_times, scaler) in enumerate(normal_and_generate_dataset_time(
        all_data_filename,
        train_rate=train_rate,
        valid_rate=valid_rate,
        recent_prior=recent_prior,
        week_prior=week_prior,
        one_day_period=one_day_period,
        days_of_week=days_of_week,
        pre_len=pre_len)):
    if False:#前100个，没写默认是false
        x = x[:100]
        y = y[:100]
        target_times = target_times[:100]
        high_x = high_x[:100]
        high_y = high_y[:100]
        high_target_times = high_target_times[:100]

    if 'nyc' in all_data_filename:
        graph_x = x[:, :, [0, 46, 47], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))#4584*7*3*400,注意reshape和交换维度的不同
        high_graph_x = high_x[:, :, [0, 46, 47], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))#1337*7*3*400
        graph_x = np.dot(graph_x, grid_node_map)#4584*7*3*243
        high_graph_x = np.dot(high_graph_x, grid_node_map)
    if 'chicago' in all_data_filename:
        graph_x = x[:, :, [0, 39, 40], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))
        high_graph_x = high_x[:, :, [0, 39, 40], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))
        graph_x = np.dot(graph_x, grid_node_map)
        high_graph_x = np.dot(high_graph_x, grid_node_map)

    print("feature:", str(x.shape), "label:", str(y.shape), "time:", str(target_times.shape),
            "high feature:", str(high_x.shape), "high label:", str(high_y.shape))
    print("graph_x:", str(graph_x.shape),
            "high_graph_x:", str(high_graph_x.shape))
    if x.shape[0] ==1080:
        b = np.ones_like(x)
        b = np.where(x>1,1,0)
        print('x的最大值:{}'.format(np.sum(b)))
        print('x的最小值:{}'.format(np.min(x)))
    if idx == 0:
        scaler = scaler
        train_data_shape = x.shape
        time_shape = target_times.shape
        graph_feature_shape = graph_x.shape
    loaders.append(Data.DataLoader(
        Data.TensorDataset(
            torch.from_numpy(x),
            torch.from_numpy(target_times),#4584*32
            torch.from_numpy(graph_x),
            torch.from_numpy(y)
        ),
        batch_size=batch_size,
        shuffle=(idx == 0)
    ))
    if idx == 2:
        high_test_loader = Data.DataLoader(
            Data.TensorDataset(
                torch.from_numpy(high_x),
                torch.from_numpy(high_target_times),
                torch.from_numpy(high_graph_x),
                torch.from_numpy(high_y)
            ),
            batch_size=batch_size,
            shuffle=(idx == 0)
        )
train_loader, val_loader, test_loader = loaders
################ 获取测试数据##################
nums_of_filter = []
for _ in range(2):
    nums_of_filter.append(gcn_num_filter)

GSNet_Model = GSNet(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
                    gru_hidden_size, time_shape[1], graph_feature_shape[2],
                    nums_of_filter, north_south_map, west_east_map)
print("模型参数---------------------------------------------")
print(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
        gru_hidden_size, time_shape[1], graph_feature_shape[2],
        nums_of_filter)

# multi gpu
if torch.cuda.device_count() > 10:
    print("Let's use", torch.cuda.device_count(), "GPUs!", flush=True)
    GSNet_Model = nn.DataParallel(GSNet_Model)
############### 载入模型######################
GSNet_Model.to(device)
# print(GSNet_Model)
GSNet_Model.load_state_dict(torch.load("./data/model_parameter.pkl"))
print("模型加载成功！")

num_of_parameters = 0
for name, parameters in GSNet_Model.named_parameters():
    num_of_parameters += np.prod(parameters.shape)
print("Number of Parameters: {}".format(num_of_parameters), flush=True)

trainer = optim.Adam(GSNet_Model.parameters(), lr=learning_rate)
early_stop = EarlyStopping(patience=patience, delta=delta)

risk_mask = get_mask(mask_filename)
road_adj = get_adjacent(road_adj_filename)
risk_adj = get_adjacent(risk_adj_filename)
if poi_adj_filename == "":
    poi_adj = None
else:
    poi_adj = get_adjacent(poi_adj_filename)


feature: (4584, 7, 48, 20, 20) label: (4584, 1, 20, 20) time: (4584, 32) high feature: (1337, 7, 48, 20, 20) high label: (1337, 1, 20, 20)
graph_x: (4584, 7, 3, 243) high_graph_x: (1337, 7, 3, 243)
feature: (1080, 7, 48, 20, 20) label: (1080, 1, 20, 20) time: (1080, 32) high feature: (315, 7, 48, 20, 20) high label: (315, 1, 20, 20)
graph_x: (1080, 7, 3, 243) high_graph_x: (315, 7, 3, 243)
x的最大值:7
x的最小值:0.0
feature: (1080, 7, 48, 20, 20) label: (1080, 1, 20, 20) time: (1080, 32) high feature: (315, 7, 48, 20, 20) high label: (315, 1, 20, 20)
graph_x: (1080, 7, 3, 243) high_graph_x: (315, 7, 3, 243)
x的最大值:3
x的最小值:0.0
模型参数---------------------------------------------
48 5 7 1 256 32 3 [64, 64]
模型加载成功！
Number of Parameters: 6278578


In [ ]:
#测试代码，功能是验证历史交通值的改变的影响
import pickle
with open('test_data.pkl','wb') as fil:
    pickle.dump(fil, fil)

In [8]:
#配置log日志输出
from myfunction import get_root_logger, logger_info
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")


2023-03-22 16:02:55,739 - INFO - Loading config file from 


In [24]:
#攻击，并写入数据,20230315导入包修改为lib.utils_new_round
import yaml
from mmcv import Config
from myfunction import log_test_results
from lib.utils_new_round import log_test_csv,node_map,prenoise_pgd,rcr_pgd,fgsm,min,kl_pgd,JS_pgd,rand_attack,nonenoise_pgd
from myfunction import get_root_logger, logger_info
#配置logo
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")
cfgin = Config.fromfile('attack_setting.yaml')
#根据配置选择攻击方法
if cfgin.attacker == 'white_attacker':
    header = ['dataset', 'model','node_select', 'method', 'K','batch size', 
              'clean_RMSE', 'clean_recall', 'clean_MAP', 'clean_RCR',
              'adv_RMSE', 'adv_recall', 'adv_MAP', 'adv_RCR',
              'local_adv_RMSE', 'local_adv_recall', 'local_adv_MAP', 'local_adv_RCR','备注信息：日期，事故，天气都修改了']
    #文件名
    file_name = 'grey_box-data-{}_num-nodes{}_eps{}-model-{}'.format(
        cfgin.dataset, cfgin.test_attack_nodes, cfgin.test_epsilon, cfgin.backbone)
    log_test_results(cfgin.result_dir, header, file_name)
    #攻击方法+节点选择组合
    for i in cfgin.select_node:
        for j in cfgin.attack_method:
            logger.info(i+'_'+j)
            parm_att = j #攻击方法的函数名称
            parm_node = i #节点选择的方法
            #进行攻击,输出攻击日志
            # 1. 节点选择，返回攻击的节点
            ack_map = node_map(i, cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                grid_node_map, device, data_type='nyc')
            # 2. 攻击方法选择，返回未攻击后的预测值，标签值,攻击前的预测值，攻击过程打印每个信息
            adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                    grid_node_map, scaler, risk_mask, device, data_type='nyc')
            #计算结果并打印、存储,注意表头信息需要传递参数
            log_test_csv(adv_val_predict, val_target, val_predict,cfgin,i,j,file_name,risk_mask)
            torch.cuda.empty_cache()
           
            



2023-03-15 19:44:21,473 - INFO - Loading config file from 
2023-03-15 19:44:21,477 - INFO - random_kl_pgd


dataset,model,node_select,method,K,batch size,clean_RMSE,clean_recall,clean_MAP,clean_RCR,adv_RMSE,adv_recall,adv_MAP,adv_RCR,local_adv_RMSE,local_adv_recall,local_adv_MAP,local_adv_RCR


2023-03-15 19:44:25,549 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 6.9147 adv_Recall: 30.2405 adv_MAP: 0.1479 adv_RCR: 0.7603 time:1.963
2023-03-15 19:44:29,506 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 8.2478 adv_Recall: 33.9450 adv_MAP: 0.1686 adv_RCR: 0.6849 time:1.954
2023-03-15 19:44:33,502 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 8.1155 adv_Recall: 30.0781 adv_MAP: 0.1838 adv_RCR: 0.6990 time:1.975
2023-03-15 19:44:37,489 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 7.3410 adv_Recall: 38.5870 adv_MAP: 0.1801 adv_RCR: 0.6760 time:1.977
2023-03-15 19:44:41,450 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

NYC,Gsnet,random,kl_pgd,80,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,7.609718393336176,30.86871644704734,0.15709620075475528,0.7076626931170686,0.47499106800314655,99.7314806121491,0.996674340736373,0.0007441392842060441


2023-03-15 19:45:39,674 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 6.9223 adv_Recall: 33.3333 adv_MAP: 0.1738 adv_RCR: 0.7193 time:1.957
2023-03-15 19:45:43,644 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 8.1514 adv_Recall: 35.3211 adv_MAP: 0.1524 adv_RCR: 0.6903 time:1.956
2023-03-15 19:45:47,601 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 8.0789 adv_Recall: 29.6875 adv_MAP: 0.1717 adv_RCR: 0.6931 time:1.955
2023-03-15 19:45:51,550 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 7.3094 adv_Recall: 37.7717 adv_MAP: 0.1863 adv_RCR: 0.6936 time:1.956
2023-03-15 19:45:55,485 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

...

NYC,Gsnet,sum_yi_loss,prenoise_pgd,80,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,7.7266553173140915,30.17325524646169,0.15225431386388852,0.7157177900744007,0.33367835631402765,99.63008168247113,0.9952437034869002,0.001048770427896875
(1080, 20, 20)


2023-03-15 19:56:47,628 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 7.0562 adv_Recall: 32.3024 adv_MAP: 0.1532 adv_RCR: 0.7354 time:1.972
2023-03-15 19:56:51,588 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 8.4328 adv_Recall: 34.8624 adv_MAP: 0.1480 adv_RCR: 0.6895 time:1.963
2023-03-15 19:56:55,518 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 8.2207 adv_Recall: 28.5156 adv_MAP: 0.1720 adv_RCR: 0.7027 time:1.951
2023-03-15 19:56:59,501 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 7.3861 adv_Recall: 36.6848 adv_MAP: 0.1691 adv_RCR: 0.6842 time:1.979
2023-03-15 19:57:03,472 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

NYC,Gsnet,sum_yi_loss_test,kl_pgd,80,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,7.7383810746495385,30.234260614934115,0.15293442478273017,0.7124594420611071,0.3238117121826967,99.60754858698714,0.9951618626723833,0.0010628301472911138
(1080, 20, 20)


2023-03-15 19:58:14,230 - INFO - Info:  [64/1080 (6%)]	  clean_RMSE: 7.1454 clean_Recall: 35.7388 clean_MAP: 0.1823 clean_RCR: 0.6776   adv_RMSE: 7.0378 adv_Recall: 32.6460 adv_MAP: 0.1549 adv_RCR: 0.7367 time:1.947
2023-03-15 19:58:18,166 - INFO - Info:  [128/1080 (12%)]	  clean_RMSE: 8.5350 clean_Recall: 38.0734 clean_MAP: 0.1698 clean_RCR: 0.6284   adv_RMSE: 8.3845 adv_Recall: 33.9450 adv_MAP: 0.1410 adv_RCR: 0.6960 time:1.947
2023-03-15 19:58:22,123 - INFO - Info:  [192/1080 (18%)]	  clean_RMSE: 8.2390 clean_Recall: 30.0781 clean_MAP: 0.1902 clean_RCR: 0.6865   adv_RMSE: 8.1928 adv_Recall: 28.5156 adv_MAP: 0.1750 adv_RCR: 0.7024 time:1.953
2023-03-15 19:58:26,078 - INFO - Info:  [256/1080 (24%)]	  clean_RMSE: 7.3895 clean_Recall: 40.7609 clean_MAP: 0.2031 clean_RCR: 0.6433   adv_RMSE: 7.3718 adv_Recall: 36.6848 adv_MAP: 0.1708 adv_RCR: 0.6808 time:1.963
2023-03-15 19:58:30,014 - INFO - Info:  [320/1080 (29%)]	  clean_RMSE: 7.9974 clean_Recall: 26.4550 clean_MAP: 0.1315 clean_RCR: 0

NYC,Gsnet,sum_yi_loss_test,prenoise_pgd,80,32,7.8062616694831215,33.24792581747194,0.178880014328625,0.6745602564431761,7.708861901658873,30.30746705710102,0.15511572537217796,0.7118845999724102,0.33170909960990125,99.60191531311614,0.9951011296805053,0.0010803891220462123


In [15]:
torch.cuda.empty_cache()